- Preparar cuenta y CLI (suscripción activa). 

- Crear Resource Group.

- Crear VNet + subredes (y NSG básico). 


- Crear Key Vault (modelo RBAC recomendado) y ajustar permisos. 


- Crear grupos en Entra ID y asignar roles (acceso a Key Vault / RG). 


- Desplegar VM en la VNet, habilitar Managed Identity y dar permiso al identity sobre Key Vault (prueba). 

- Opcional avanzado: proteger Key Vault con Private Endpoint (Private Link).

- az login -> inciar sesión en AZure CLI
- az account list --output table ->

### crear un grupo de recursos para hospedar la red virtual:
- az group create --name rg-data-lab --location eastus
- az group list --output table -> listar los grupos de recursos creados

#### Crear una VNet (red virtual) y una subred en el grupo de recursos gr-data-lab:
- az network vnet create --resource-group rg-data-lab --name vnet-data-lab --address-prefixes 10.10.0.0/16 --subnet-name subnet-app --subnet-prefixes 10.10.1.0/24 --location eastus

#### verificar lista de VNet y subred creadas en el recurso 
- az network vnet list --resource-group rg-data-lab --output table
- az network vnet subnet list --resource-group rg-data-lab --vnet-name vnet-data-lab --output table

#### Crear un Network Security Group (NSG)

Crear un Network Security Group (NSG) y asociarlo a tu subred te permite controlar el tráfico de red (inbound/outbound), actuando como un firewall a nivel de red dentro de Azure.

- az network nsg create --resource-group rg-data-lab --name nsg-app

#### listar/ ver detalles de los NSG 
 - az network nsg list --resource-group rg-data-lab --output table

 - az network nsg show --resource-group rg-data-lab --name nsg-app --output json

 - az network nsg show --resource-group rg-data-lab --name nsg-app --output table

#### asociar una nsg a un subred
 az network vnet subnet update --resource-group rg-data-lab --vnet-name vnet-data-lab --name subnet-app --network-security-group nsg-app

### Crear Key Vault (modelo RBAC recomendado)
¡¡¡el nombre del Key Vault debe ser único a nivel global (solo letras y números, sin mayúsculas)!!! 

-- se debe registrar el proveedor:
- az provider register --namespace Microsoft.KeyVault
- az provider show --namespace Microsoft.KeyVault --query "registrationState" --output tsv

ahora si se pueden crer las key vault:

-  az keyvault create --name kv-datalab2025 --resource-group rg-data-lab --location eastus --sku standard

#### listar las key vault existentes
- az keyvault list --resource-group rg-data-lab --output table
- az keyvault show --name kv-datalab2025 --resource-group rg-data-lab -o table

#### asigar un rol a un usuario
obtener el id en Entra:
az ad signed-in-user show --query id -o tsv

#### crear la asignacion
 az role assignment create --assignee da32158b-55e2-429d-80ea-8369b5ed768e --role "Key Vault Secrets Officer" --scope $(az keyvault show --name kv-datalab2025 --query id -o tsv)

#### Guardar secretos en las key vault 
 az keyvault secret set \
  --vault-name kv-datalab2025 \
  --name "DbPassword" \
  --value "SuperP@ssw0rd123"

Azure RBAC (Role-Based Access Control) es el sistema de control de acceso basado en roles de Azure.
Sirve para definir quién puede hacer qué sobre qué recurso dentro de tu suscripción de Azure.

Es decir:

Quién = un usuario, grupo, aplicación o servicio administrado.

Qué puede hacer = acciones (leer, escribir, eliminar, administrar).

Sobre qué = un recurso (ej. un Key Vault, un Storage, una VM).

#### Crear Grupos en Microsoft Entra ID

- az ad group create --display-name "Lab_DataAnalytics" --mail-nickname "lab_dataanalytics"

#### Obtener el Object ID de cada usuario
- az ad user show --id jpmiranda@imsabogado.com --query id -o tsv
- az ad user show --id crueda@imsabogado.com --query id -o tsv
- az ad user show --id apino@rpsprocessing.com --query id -o tsv

#### agregar miembros al grupo
- az ad group member add --group "Lab_DataAnalytics" --member-id a6ae23c8-6387-47ee-ba58-ef448b269ee6


#### listar miembros del grupo
az ad group member list \
  --group "Lab_DataAnalytics" \
  --output table

#### Dar acceso al Key Vault usando RBAC
Dar al grupo Lab_DataAnalytics el rol Key Vault Secrets User

#### obtener el Object ID del grupo
az ad group show --group "Lab_DataAnalytics" --query id -o tsv
7eedd430-6e28-49f8-8a75-4636731fc311

#### asignar rol por Object ID
az role assignment create \
  --assignee <group-object-id> \
  --role "Key Vault Secrets User" \
  --scope $(az keyvault show --name kv-datalab2025 --query id -o tsv)

#### Desplegar una VM en la VNet y habilitar Managed Identity

#### Crear VM (ejemplo Linux) en la subred subnet-app

### maquina virtual vm-datalab-1

- az vm create --resource-group rg-data-lab --name vm-datalab-1 --image Ubuntu2404 --size Standard_B1s --admin-username azureuser --generate-ssh-keys --vnet-name vnet-data-lab --subnet subnet-app

### maquina virtual vm-datalab-2
-  az vm create --resource-group rg-data-lab --name vm-datalab-2 --image Ubuntu2404 --size Standard_B1s --admin-username azureuser --generate-ssh-keys --vnet-name vnet-data-lab --subnet subnet-app


#### ver detalles de una maquina virtual creada:
- az vm show --resource-group rg-data-lab --name vm-datalab-1 --show-details --output table
-  az vm list --show-details --output table

#### apgar la maquina virtual y liberar los recursos de coómputo
- az vm deallocate --resource-group rg-data-lab --name vm-datalab-1
### encender la maquina virtual
- az vm start --resource-group rg-data-lab --name vm-datalab-1

#### Implementación de Azure Bastion
- Bastion es un servicio PaaS administrado por Azure
- usar el explorador para conectarse a máquinas virtuales de la red virtual a través de Secure Shell (SSH) o protocolo de escritorio remoto (RDP) mediante sus direcciones IP privadas.

#### Crear una subred de Bastion en la red virtual

- Esta sub red es reservada exclusivamente para los recursos de Azure bastion y debe llamarse ¡¡¡AzureBastionSubnet!!!

- az network vnet subnet create --name AzureBastionSubnet --resource-group rg-data-lab --vnet-name vnet-data-lab --address-prefixes 10.10.3.0/27

#### crear un ip pública para Bastion
Esta dirección se usa para conectarse al hos bastion desde internet. se debe crear en el mismo grupo de recursos.

- az network public-ip create --resource-group rg-data-lab --name public-ip --sku Standard --location eastus --zone 1 2 3

#### listar las ip publicas creadas

-  az network public-ip list --resource-group rg-data-lab --output table

#### crear el bastion

- az network bastion create --name bastion-lab --public-ip-address public-ip --resource-group rg-data-lab --vnet-name vnet-data-lab --location eastus --sku Standard --enable-tunneling true 

#### listar bastiones creado
az network bastion list \
  --resource-group rg-data-lab \
  --output table

#### Análsis de Maquinas virtuales 

- az vim list --show-details --output table 

#### lsitas las maquinas virtuales y ver su ID 
se utiliza para realizar la conexión

az vm show --resource-group rg-data-lab --name vm-datalab-1 --query "id" --output tsv

/subscriptions/db059ab1-ccee-408a-9ad2-8e1d6fddddf4/resourceGroups/rg-data-lab/providers/Microsoft.Compute/virtualMachines/vm-datalab-1

#### conectarse a la maquina virtual
az network bastion ssh `
  --name bastion-lab `
  --resource-group rg-data-lab `
  --target-resource-id $(az vm show -g rg-data-lab -n vm-datalab-1 --query id -o tsv) `
  --auth-type ssh-key `
  --username azureuser `
  --ssh-key $env:USERPROFILE\.ssh\id_rsa
 #### se requiere la extensión de ssh instalada en cli 
 az extensión add -n ssh 

#### RESUMEN DE GRUPO DE RECURSOS

#### grupo de recursos
- az group list 
rg-data-lab       eastus      Succeeded

#### vnet 
- az network vnet list --resource-group rg-data-lab --output table

vnet-data-lab  rg-data-lab      eastus      2             10.10.0.0/16                False

#### subnet
-  az network vnet subnet list --resource-group rg-data-lab --vnet-name vnet-data-lab --output table

10.10.3.0/27     AzureBastionSubnet  Disabled                          Enabled                              Succeeded            rg-data-lab

10.10.1.0/24     subnet-app          Disabled                          Enabled                              Succeeded            rg-data-lab

#### nsg
-  az network nsg show --resource-group rg-data-lab --name nsg-app --output table

eastus      nsg-app  Succeeded            rg-data-lab      2e58ebc6-7cdd-4f17-83de-ebcbf32f8b47


HACER UNA TABLA hubspot
 record ID  contacto y negocios llave única
 asosiation ID


